In [3]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import json

In [2]:
obj = """
{"name": "Wes",
"places_lived": ["United Status", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
                {"name": "Katie", "age": 33, "pet": "Cisco"}]

}
"""

In [4]:
result = json.loads(obj)

In [5]:
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United Status', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [6]:
asjson = json.dumps(result)

In [7]:
asjson

'{"name": "Wes", "places_lived": ["United Status", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Katie", "age": 33, "pet": "Cisco"}]}'

In [8]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])

In [9]:
siblings

,name,age
0,Scott,25
1,Katie,33


In [18]:
from lxml import html
import requests

In [19]:
page = requests.get('http://finance.yahoo.com/q/op?s=AAPL+Options')

In [33]:
tree = html.fromstring(page.text)

<Element html at 0xb3b85e8>

In [41]:
links = tree.findall('.//a')

In [42]:
links[15:20]

[<Element a at 0xb3b8f48>,
 <Element a at 0xb3b8f98>,
 <Element a at 0xb3b5048>,
 <Element a at 0xb3b5098>,
 <Element a at 0xb3b50e8>]

In [43]:
lnk = links[28]

In [44]:
lnk

<Element a at 0xb3b53b8>

In [45]:
lnk.get('href')

'/quote/NFLX?p=NFLX'

In [61]:
links[29].get('href')

'/quote/TSLA?p=TSLA'

In [47]:
lnk.text_content()

'NFLX'

In [48]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]

In [49]:
urls[-10:]

['https://smallbusiness.yahoo.com',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US&id=SLN2310',
 'http://help.yahoo.com/l/us/yahoo/finance/',
 'https://yahoo.uservoice.com/forums/382977',
 'http://info.yahoo.com/privacy/us/yahoo/',
 'http://info.yahoo.com/relevantads/',
 'http://info.yahoo.com/legal/us/yahoo/utos/utos-173.html',
 'http://twitter.com/YahooFinance',
 'http://facebook.com/yahoofinance',
 'http://yahoofinance.tumblr.com']

In [62]:
tables = doc.findall('.//table')
tables

[<Element table at 0xb3bfe58>,
 <Element table at 0xb3bfcc8>,
 <Element table at 0xb3bff48>]

In [54]:
calls = tables[1]

In [55]:
puts = tables[2]

In [56]:
rows = calls.findall('.//tr')

In [57]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [58]:
_unpack(rows[0], kind='th')

['Strike',
 'Contract Name',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [59]:
_unpack(rows[1], kind='td')

['2.50',
 'AAPL170519C00002500',
 '153.15',
 '151.95',
 '154.40',
 '8.45',
 '5.84%',
 '4',
 '0',
 '2,237.50%']

In [63]:
from pandas.io.parsers import TextParser

In [65]:
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()
    

In [66]:
call_data = parse_options_data(calls)

In [67]:
put_data = parse_options_data(puts)

In [68]:
call_data[:10]

,Strike,Contract Name,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,2.5,AAPL170519C00002500,153.15,151.95,154.40,8.45,5.84%,4,0,"2,237.50%"
1,5.0,AAPL170519C00005000,150.65,149.45,151.65,0.00,0.00%,4,0,"1,562.50%"
2,7.5,AAPL170519C00007500,148.05,146.95,149.30,0.00,0.00%,3,0,"1,478.13%"
3,10.0,AAPL170519C00010000,143.53,145.65,146.60,0.00,0.00%,10,0,"1,665.63%"
4,20.0,AAPL170519C00020000,133.52,135.65,136.60,0.00,0.00%,10,0,"1,210.94%"
5,35.0,AAPL170519C00035000,96.80,96.85,97.60,0.00,0.00%,4,0,0.00%
6,50.0,AAPL170519C00050000,103.10,105.65,106.60,0.00,0.00%,600,0,676.17%
7,70.0,AAPL170519C00070000,82.94,85.65,86.60,0.00,0.00%,222,0,489.84%
8,80.0,AAPL170519C00080000,73.42,75.70,76.60,0.00,0.00%,5,0,419.34%
9,85.0,AAPL170519C00085000,68.45,70.70,71.60,0.00,0.00%,3,0,385.55%
